In [ ]:
import pandas as pd
import csv
from datetime import datetime
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def combine_rpg_reviews(scraped_csv_path, steam_export_csv_path, output_csv_path=None):
    """
    Combine RPG reviews from scraped data and Steam export data
    """

    try:
        logger.info("Reading scraped RPG reviews data...")
        scraped_df = pd.read_csv(scraped_csv_path)
        logger.info(f"Loaded {len(scraped_df)} reviews from scraped data")
        rpg_game_names = set(scraped_df['game_name'].unique())
        logger.info(f"Found {len(rpg_game_names)} unique RPG games in scraped data")
        logger.info("Reading Steam Game Reviews export data...")
        steam_df = pd.read_csv(steam_export_csv_path)
        logger.info(f"Loaded {len(steam_df)} reviews from Steam export")
        logger.info("Filtering Steam export for RPG games...")
        steam_rpg_df = steam_df[steam_df['game_name'].isin(rpg_game_names)]
        logger.info(f"Found {len(steam_rpg_df)} RPG reviews in Steam export")

        scraped_formatted = scraped_df.copy()
        scraped_formatted['review'] = scraped_formatted['review_text']
        scraped_formatted['source'] = 'scraped'
        steam_formatted = steam_rpg_df[['review', 'hours_played', 'recommendation', 'game_name']].copy()
        steam_formatted['genre'] = 'RPG'
        steam_formatted['source'] = 'steam_export'

        logger.info("Combining datasets...")

        final_columns = ['game_name', 'hours_played', 'genre', 'review', 'recommendation', 'source']

        scraped_final = scraped_formatted[['game_name', 'hours_played', 'genre', 'review', 'recommendation', 'source']]
        steam_final = steam_formatted[['game_name', 'hours_played', 'genre', 'review', 'recommendation', 'source']]

        combined_df = pd.concat([scraped_final, steam_final], ignore_index=True)

        combined_df = combined_df.dropna(subset=['review'])
        combined_df = combined_df[combined_df['review'].str.strip() != '']

        if not output_csv_path:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_csv_path = f"combined_rpg_reviews_{timestamp}.csv"

        combined_df.to_csv(output_csv_path, index=False, encoding='utf-8')
        logger.info(f"Combined dataset saved to {output_csv_path}")

        print(f"\n=== COMBINATION SUMMARY ===")
        print(f"Total combined reviews: {len(combined_df)}")
        print(f"Reviews from scraped data: {len(scraped_final)}")
        print(f"Reviews from Steam export: {len(steam_final)}")
        print(f"Unique games: {len(combined_df['game_name'].unique())}")

        rec_counts = combined_df['recommendation'].value_counts()
        print(f"\n=== RECOMMENDATION DISTRIBUTION ===")
        for rec, count in rec_counts.items():
            percentage = (count / len(combined_df)) * 100
            print(f"{rec}: {count} ({percentage:.1f}%)")

        source_counts = combined_df['source'].value_counts()
        print(f"\n=== SOURCE DISTRIBUTION ===")
        for source, count in source_counts.items():
            percentage = (count / len(combined_df)) * 100
            print(f"{source}: {count} ({percentage:.1f}%)")

        game_counts = combined_df['game_name'].value_counts()
        print(f"\n=== TOP 10 GAMES BY REVIEW COUNT ===")
        for game, count in game_counts.head(10).items():
            print(f"{game}: {count} reviews")

        scraped_games = set(scraped_df['game_name'].unique())
        steam_games = set(steam_rpg_df['game_name'].unique())
        common_games = scraped_games.intersection(steam_games)

        print(f"\n=== OVERLAP ANALYSIS ===")
        print(f"Games only in scraped data: {len(scraped_games - steam_games)}")
        print(f"Games only in Steam export: {len(steam_games - scraped_games)}")
        print(f"Games in both datasets: {len(common_games)}")

        if common_games:
            print(f"\nGames found in both datasets:")
            for game in sorted(common_games):
                scraped_count = len(scraped_df[scraped_df['game_name'] == game])
                steam_count = len(steam_rpg_df[steam_rpg_df['game_name'] == game])
                print(f"  {game}: {scraped_count} (scraped) + {steam_count} (export) = {scraped_count + steam_count} total")

        return combined_df, output_csv_path

    except Exception as e:
        logger.error(f"Error combining CSV files: {e}")
        raise

def main():
    """
    Main function to run the CSV combination
    """

    scraped_csv_path = "C:/Users/0/steam_rpg_reviews_20250623_171010.csv"  # Scraped data file
    steam_export_csv_path = "C:/Users/0/Downloads/Steam Game Reviews export 2025-06-23 08-11-29.csv"  # Steam export file

    print("=== RPG GAME REVIEWS CSV COMBINER ===")
    print(f"Scraped data file: {scraped_csv_path}")
    print(f"Steam export file: {steam_export_csv_path}")

    try:
        combined_df, output_file = combine_rpg_reviews(scraped_csv_path, steam_export_csv_path)
        print(f"\nCombined RPG reviews saved to: {output_file}")

    except FileNotFoundError as e:
        print(f"\nFile not found - {e}")
        print("Please make sure both CSV files exist and update the file paths in the script.")

    except Exception as e:
        print(f"\n{e}")

if __name__ == "__main__":
    main()